In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set(style="darkgrid")

In [3]:
import os
df = pd.read_excel('df_final.xlsx')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Columns: 746 entries, Unnamed: 0 to conversion_current
dtypes: datetime64[ns](3), float64(728), int64(12), object(3)
memory usage: 3.5+ MB


In [5]:
df.describe()

,Unnamed: 0,num_expon_apps,num_apps,num_expon_apps_0,num_expon_apps_1,num_expon_apps_2,num_expon_apps_3plus,num_apps_0,num_apps_1,num_apps_2,...,primary_calls,secondary_calls,target_calls_qty_T1,target_deals_qty_T1,plan_area,meetings_qty,agent_meetings_qty,deals,target_conversion_rate,conversion_current
count,620.000000,620.000000,620.000000,620.000000,620.000000,620.000000,620.000000,620.000000,620.000000,620.000000,...,614.000000,614.000000,617.000000,453.000000,615.00000,610.000000,610.000000,362.000000,453.000000,613.000000
mean,316.500000,654.816129,1388.237097,0.129032,201.487097,232.332258,113.398387,1.693548,516.812903,576.282258,...,289.203583,259.633550,454.672609,38.357616,4103.30839,425.921311,29.298361,16.640884,0.096426,inf
std,179.122863,687.928023,1117.488570,1.008588,228.681007,269.405873,141.085983,5.705731,549.074172,584.488243,...,270.035734,271.506295,413.326716,56.450818,3090.67527,356.236071,29.121615,24.125096,0.097051,NaN
min,7.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,26.50000,2.000000,0.000000,1.000000,0.000561,0.000000
25%,161.750000,138.250000,442.500000,0.000000,36.750000,42.000000,20.000000,0.000000,134.750000,124.500000,...,88.250000,61.000000,143.000000,7.000000,1555.76500,138.250000,8.000000,3.000000,0.022613,0.000000
50%,316.500000,382.000000,1211.500000,0.000000,141.000000,118.500000,60.000000,0.000000,422.500000,389.500000,...,202.000000,171.500000,325.000000,19.000000,3704.34000,321.000000,19.000000,8.000000,0.071429,0.008621
75%,471.250000,992.000000,2085.000000,0.000000,287.250000,358.750000,152.500000,0.000000,666.250000,871.500000,...,399.000000,349.250000,640.000000,41.000000,5715.55500,625.250000,43.000000,19.000000,0.144019,0.056701
max,626.000000,4429.000000,5613.000000,8.000000,2613.000000,1981.000000,1299.000000,29.000000,5107.000000,3997.000000,...,1616.000000,1633.000000,2398.000000,392.000000,17707.27000,1682.000000,198.000000,194.000000,1.000000,inf


In [6]:
df.head()

,Unnamed: 0,smt_construction_projectid,eom,num_expon_apps,num_apps,num_expon_apps_0,num_expon_apps_1,num_expon_apps_2,num_expon_apps_3plus,num_apps_0,...,target_calls_qty_T1,target_deals_qty_T1,smt_name,smt_region_name,plan_area,meetings_qty,agent_meetings_qty,deals,target_conversion_rate,conversion_current
0,7,07091659-2705-E911-80E5-005056B8D207,2022-03-31,1690,1698,0,567,803,242,0,...,918.0,NaN,АЛХИМОВО,Москва,4346.03,604.0,47.0,NaN,NaN,0.000000
1,8,07091659-2705-E911-80E5-005056B8D207,2022-04-30,1326,1686,0,486,542,194,0,...,1029.0,NaN,АЛХИМОВО,Москва,4148.49,634.0,21.0,NaN,NaN,0.000000
2,9,07091659-2705-E911-80E5-005056B8D207,2022-05-31,1187,1628,0,345,538,152,0,...,1349.0,NaN,АЛХИМОВО,Москва,2963.20,933.0,34.0,NaN,NaN,0.000000
3,10,07091659-2705-E911-80E5-005056B8D207,2022-06-30,962,1504,0,347,397,131,0,...,1633.0,2.0,АЛХИМОВО,Москва,3556.00,1134.0,48.0,NaN,0.001225,0.000000
4,11,07091659-2705-E911-80E5-005056B8D207,2022-07-31,981,1405,0,340,399,104,0,...,1429.0,NaN,АЛХИМОВО,Москва,3840.37,1442.0,66.0,1.0,NaN,0.000881


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Columns: 746 entries, Unnamed: 0 to conversion_current
dtypes: datetime64[ns](3), float64(728), int64(12), object(3)
memory usage: 3.5+ MB


In [11]:
# Сбросить опции к начальным значениям (по умолчанию)
pd.reset_option('display.max_columns')

In [13]:
# Установить опцию для отображения всех столбцов
pd.set_option('display.max_columns', None)

# Теперь когда вы вызовете функцию isnull().sum(), все столбцы будут отображены
print(df.isnull().sum())

Unnamed: 0                      0
smt_construction_projectid      0
eom                             0
num_expon_apps                  0
num_apps                        0
                             ... 
meetings_qty                   10
agent_meetings_qty             10
deals                         258
target_conversion_rate        167
conversion_current              7
Length: 746, dtype: int64


In [14]:
# Увеличить лимит вывода строк
pd.set_option('display.max_rows', None)

# Теперь когда вы вызовете функцию isnull().sum(), все строки будут отображены
print(df.isnull().sum())

Unnamed: 0                                                                            0
smt_construction_projectid                                                            0
eom                                                                                   0
num_expon_apps                                                                        0
num_apps                                                                              0
num_expon_apps_0                                                                      0
num_expon_apps_1                                                                      0
num_expon_apps_2                                                                      0
num_expon_apps_3plus                                                                  0
num_apps_0                                                                            0
num_apps_1                                                                            0
num_apps_2                      

In [15]:
from IPython.display import display

# Используйте функцию display для отображения всего DataFrame или Series.
display(df.isnull().sum())

Unnamed: 0                                                                            0
smt_construction_projectid                                                            0
eom                                                                                   0
num_expon_apps                                                                        0
num_apps                                                                              0
num_expon_apps_0                                                                      0
num_expon_apps_1                                                                      0
num_expon_apps_2                                                                      0
num_expon_apps_3plus                                                                  0
num_apps_0                                                                            0
num_apps_1                                                                            0
num_apps_2                      